# Kafka Producer

**Set Cluster Mount Point**

In [ ]:
# import os.path

# ###### Mount Point 1 through Oauth security.
# storageAccount = "gen10dbcdatalake"
# storageContainer = "diabeetus-capstone"
# clientSecret = config.clientSecret
# clientid = config.clientid
# mount_point = "/mnt/eddydoering/diabeetus"



# configs = {"fs.azure.account.auth.type": "OAuth",
#        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#        "fs.azure.account.oauth2.client.id": clientid,
#        "fs.azure.account.oauth2.client.secret": clientSecret,
#        "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
#        "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

# try: 
#     dbutils.fs.unmount(mount_point)
# except:
#     pass

# dbutils.fs.mount(
# source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/",
# mount_point = mount_point,
# extra_configs = configs)

**Read Raw CSV as list of dictionaries, where each row is one dictionary**

In [ ]:
import csv
list_dict = spark.read.csv("/mnt/eddydoering/diabeetus/diabetic_data.csv", header = "true").toPandas().to_dict(orient="records")

**Error Functions**

In [ ]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

**Setup Producer Parameters**

In [ ]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
from confluent_kafka.admin import AdminClient, NewTopic
import config               #hidden config file with keys and login credentials


#KAFKA variables
confluentClusterName = config.confluentClusterName
confluentBootstrapServers = config.confluentBootstrapServers
confluentTopicName = config.confluentTopicName
schemaRegistryUrl = config.schemaRegistryUrl
confluentApiKey = config.confluentApiKey
confluentSecret = config.confluentSecret
confluentRegistryApiKey = config.confluentRegistryApiKey
confluentRegistrySecret = config.confluentRegistrySecret

admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})


#Kakfa Class Setup.
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

**Run Producer**

In [ ]:
import json

for item in list_dict:
    p.produce("diabetesfinal-10", json.dumps(item))
    p.flush()
    sleep(0.25)